In [1]:
#Function Matching Pursuit : 
#Signal Y, Dictionary D with normalized columns, number of non nul coefficients N-> list of coefficient and corresponding indices of a representation of Y in D
function MatchingPursuit(Y,D,N)
    R = Y #Residual
    Dind = [i for i = 1:length(D[1,:])] #indices of non used columns of D
    Coefs = [0] #Coefficients of Y in D
    indices = [0] # indices correspondind to the coefficients of Y in D
    for k = 1:N
        a = -1
        ind = -1
        for i in Dind #Find the column g of D such as the inner product between R and g is maximum
            innerProduct = R'*D[:,i] #warning : R and D[i,:] must be column vectors
            if abs(innerProduct) > a
                a = innerProduct
                ind = i
            end
        end
        Dind = filter!(x->x≠ind,Dind)
        R = Y - a.*D[:,ind]
        Coefs = [Coefs a]
        indices = [indices ind]
    end
    return Coefs[2:end]', indices[2:end]'
end

MatchingPursuit (generic function with 1 method)

In [2]:
Y = [1;2]
D = [[1;0] [0;1] (2^(-0.5)).*[1;1]]

coefs, indices = MatchingPursuit(Y,D,2)

([2.121320343559643 0.4999999999999998], [3 2])

In [14]:
using LinearAlgebra
print(norm(coefs),"   ",norm(Y))

2.179449471770337   2.23606797749979

In [15]:
function init_dictionary(n::Int, K::Int)
    # D must be a full-rank matrix
    D = rand(n, K)
    while rank(D) != min(n, K)
        D = rand(n, K)
    end

    @inbounds for k in 1:K
        D[:, k] ./= norm(@view(D[:, k]))
    end
    return D
end

init_dictionary (generic function with 1 method)

In [54]:
using DataStructures
using SparseArrays

# The implementation is referencing the wikipedia page
# https://en.wikipedia.org/wiki/Matching_pursuit#The_algorithm

const default_max_iter = 20
const default_tolerance = 1e-6


function SparseArrays.sparsevec(d::DefaultDict, m::Int)
    SparseArrays.sparsevec(collect(keys(d)), collect(values(d)), m)
end


function matching_pursuit_(data::AbstractVector, dictionary::AbstractMatrix,
                           max_iter::Int, tolerance::Float64)
    n_atoms = size(dictionary, 2)

    residual = copy(data)

    xdict = DefaultDict{Int, Float64}(0.)
    for i in 1:max_iter
        if norm(residual) < tolerance
            return sparsevec(xdict, n_atoms)
        end

        # find an atom with maximum inner product
        products = dictionary' * residual
        _, maxindex = findmax(abs.(products))
        maxval = products[maxindex]
        atom = dictionary[:, maxindex]

        # c is the length of the projection of data onto atom
        a = maxval / sum(abs2, atom)  # equivalent to maxval / norm(atom)^2
        residual -= atom * a

        xdict[maxindex] += a
    end
    return sparsevec(xdict, n_atoms)
end


"""
    matching_pursuit(data::Vector, dictionary::AbstractMatrix;
                     max_iter::Int = $default_max_iter,
                     tolerance::Float64 = $default_tolerance)
Find ``x`` such that ``Dx = y`` or ``Dx ≈ y`` where y is `data` and D is `dictionary`.
```
# Arguments
* `max_iter`: Hard limit of iterations
* `tolerance`: Exit when the norm of the residual < tolerance
```
"""
function matching_pursuit(data::AbstractVector, dictionary::AbstractMatrix;
                          max_iter::Int = default_max_iter,
                          tolerance = default_tolerance)

    if tolerance <= 0
        throw(ArgumentError("`tolerance` must be > 0"))
    end

    if max_iter <= 0
        throw(ArgumentError("`max_iter` must be > 0"))
    end

    if size(data, 1) != size(dictionary, 1)
        throw(ArgumentError(
            "Dimensions must match: `size(data, 1)` and `size(dictionary, 1)`."
        ))
    end

    matching_pursuit_(data, dictionary, max_iter, tolerance)
end


"""
    matching_pursuit(data::AbstractMatrix, dictionary::AbstractMatrix;
                     max_iter::Int = $default_max_iter,
                     tolerance::Float64 = $default_tolerance)
Find ``X`` such that ``DX = Y`` or ``DX ≈ Y`` where Y is `data` and D is `dictionary`.
```
# Arguments
* `max_iter`: Hard limit of iterations
* `tolerance`: Exit when the norm of the residual < tolerance
```
"""
function matching_pursuit(data::AbstractMatrix, dictionary::AbstractMatrix;
                          max_iter::Int = default_max_iter,
                          tolerance::Float64 = default_tolerance)
    K = size(dictionary, 2)
    N = size(data, 2)

    X = spzeros(K, N)

    for i in 1:N
        X[:, i] = matching_pursuit(
            vec(data[:, i]),
            dictionary,
            max_iter = max_iter,
            tolerance = tolerance
        )
    end
    return X
end

matching_pursuit

In [150]:
function K_SVD(Y,niter_KSVD,n_atoms,niter_MP)
    D = init_dictionary(size(Y,1),n_atoms)
    X = matching_pursuit(Y,D)
    for i =1:niter_KSVD
        X = matching_pursuit(Y,D)
        for k=1:n_atoms
            Xk = X[k,:]
            all(iszero,Xk)&&continue
            wk = findall(!iszero,Xk)
            indices = [j for j=1:size(D,2) if j!=k]
            Ek = Y - D[:,indices]*X[indices,:]
            Ωk = sparse(wk,1:length(wk),ones(length(wk)),size(Y,2),length(wk))
            U, S, V= svd(Ek*Ωk, full=true)
            D[:,k]=U[:,1]
            X[k,wk] = V[:,1]*S[1]
        end
    end
    return D,X
end

K_SVD (generic function with 1 method)

In [154]:
Y = [1 2 3;2 2 3]
#D = [[1;0] [0;1] (2^(-0.5)).*[1;1]]

Dic, X = K_SVD(Y,20,20,10)

([0.6548226604063008 0.7705152295190294 … -0.6804577430973588 0.6759077643516734; 0.7557825635845368 0.6374215881810387 … -0.7327873223922813 0.7369862238123062], 
  [4 , 1]  =  2.23604
  [16, 1]  =  0.011498
  [17, 1]  =  -0.00887584
  [16, 2]  =  -0.0952605
  [17, 2]  =  0.104249
  [19, 2]  =  -2.82649
  [16, 3]  =  -0.142891
  [17, 3]  =  0.156373
  [19, 3]  =  -4.23974)